In [1]:
import sys
sys.path.append('..')

In [2]:
import os
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP/pythonbooks'

In [3]:
os.chdir('..')
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP'

In [4]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


# Хочется вытащить все в питонбук, потому что из консоли работать неудобно

# загрузка данных

In [5]:
# # uncomment to install
# # add whatelse u want

!pip install ujson
!pip install tensorboardX
!pip install spacy
# 

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!python3 setup.py

You are using pip version 19.0.2, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
^C
Traceback (most recent call last):
  File "setup.py", line 386, in <module>
    download(args_)
  File "setup.py", line 71, in download
    run(['python', '-m', 'spacy', 'download', 'en'])
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/subprocess.py", line 425, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/subprocess.py", line 855, in communicate
    self.wait()
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/subprocess.py", line 1477, in wait
    (pid, sts) = self._try_wait(0)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/subprocess.py", line 1424, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt


# Обучение

In [5]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data
import util

from args import get_train_args
from collections import OrderedDict
from json import dumps
from models import BiDAF, BiDAF_CBE
from tensorboardX import SummaryWriter
from tqdm import tqdm
from ujson import load as json_load
from util import collate_fn, SQuAD

In [6]:
train_args = type('', (), {})() # empty "structure" where you can add fields

train_args.train_record_file = './data/train.npz'
train_args.dev_record_file = './data/dev.npz'
train_args.test_record_file = './data/test.npz'
train_args.word_emb_file = './data/word_emb.json'
train_args.char_emb_file = './data/char_emb.json'
train_args.train_eval_file = './data/train_eval.json'
train_args.dev_eval_file = './data/dev_eval.json'
train_args.test_eval_file = './data/test_eval.json'

train_args.name = 'train'       # ???
train_args.max_ans_len = 15     # Maximum length of a predicted answer.
train_args.num_workers = 4      # Number of sub-processes to use per data loader.
train_args.save_dir = './save/' # Base directory for saving information.
train_args.batch_size = 64      # Batch size per GPU. Scales automatically when \
                                #    multiple GPUs are available.
train_args.use_squad_v2 = True  # Whether to use SQuAD 2.0 (unanswerable) questions.
train_args.hidden_size = 100    # Number of features in encoder hidden layers
train_args.num_visuals = 10     # Number of examples to visualize in TensorBoard.
train_args.load_path = None     # Path to load as a model checkpoint.

train_args.eval_steps = 50000   # Number of steps between successive evaluations
train_args.lr = 0.4             # learning rate
train_args.l2_wd = 0.0          # L2 weight decay.
train_args.num_epochs = 30      # Number of epochs for which to train. Negative means forever.
train_args.drop_prob = 0.2      # Probability of zeroing an activation in dropout layers.
train_args.metric_name = 'F1'   # choices=('NLL', 'EM', 'F1'), name of dev metric to determine best checkpoint.
train_args.max_checkpoints = 5  # Maximum number of checkpoints to keep on disk.
train_args.max_grad_norm = 5.0  # Maximum gradient norm for gradient clipping.
train_args.seed = 224           # Random seed
train_args.ema_decay = 0.999    # Decay rate for exponential moving average of parameters.

if train_args.metric_name == 'NLL':
    # Best checkpoint is the one that minimizes negative log-likelihood
    train_args.maximize_metric = False
elif train_args.metric_name in ('EM', 'F1'):
    # Best checkpoint is the one that maximizes EM or F1
    train_args.maximize_metric = True

In [7]:
# from train.py

def main(args):
    # Set up logging and devices
    args.save_dir = util.get_save_dir(args.save_dir, args.name, training=True)
    log = util.get_logger(args.save_dir, args.name)
    tbx = SummaryWriter(args.save_dir)
    device, args.gpu_ids = util.get_available_devices()
    log.info('Args: {}'.format(dumps(vars(args), indent=4, sort_keys=True)))
    args.batch_size *= max(1, len(args.gpu_ids))

    # Set random seed
    log.info('Using random seed {}...'.format(args.seed))
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    # Get embeddings
    log.info('Loading embeddings...')
    word_vectors = util.torch_from_json(args.word_emb_file)

    
    
    # Get data loader
    log.info('Building dataset...')
    train_dataset = SQuAD(args.train_record_file, args.use_squad_v2)
    train_loader = data.DataLoader(train_dataset,
                                   batch_size=args.batch_size,
                                   shuffle=True,
                                   num_workers=args.num_workers,
                                   collate_fn=collate_fn)
    dev_dataset = SQuAD(args.dev_record_file, args.use_squad_v2)
    dev_loader = data.DataLoader(dev_dataset,
                                 batch_size=args.batch_size,
                                 shuffle=False,
                                 num_workers=args.num_workers,
                                 collate_fn=collate_fn)
    
    
    # Get model
    log.info('Building model...')
    model = BiDAF_CBE(word_vectors=word_vectors,
                      hidden_size=args.hidden_size,
                      drop_prob=args.drop_prob,
                      char_dict_size=1400,#train_dataset.get_char_dict_size(),
                      char_emb_size=args.hidden_size,
                      conv_kernel_size=3,
                      conv_depth1=50,
                      conv_output_hidden_size=args.hidden_size)
#     model.summary()
    model = nn.DataParallel(model, args.gpu_ids)
    if args.load_path:
        log.info('Loading checkpoint from {}...'.format(args.load_path))
        model, step = util.load_model(model, args.load_path, args.gpu_ids)
    else:
        step = 0
    model = model.to(device)
    model.train()
   
    ema = util.EMA(model, args.ema_decay)

    # Get saver
    saver = util.CheckpointSaver(args.save_dir,
                                 max_checkpoints=args.max_checkpoints,
                                 metric_name=args.metric_name,
                                 maximize_metric=args.maximize_metric,
                                 log=log)

    # Get optimizer and scheduler
    optimizer = optim.Adadelta(model.parameters(), args.lr,
                               weight_decay=args.l2_wd)
    scheduler = sched.LambdaLR(optimizer, lambda s: 1.)  # Constant LR

    

    # Train
    log.info('Training...')
    steps_till_eval = args.eval_steps
    epoch = step // len(train_dataset)
    while epoch != args.num_epochs:
        epoch += 1
        log.info('Starting epoch {}...'.format(epoch))
        with torch.enable_grad(), \
                tqdm(total=len(train_loader.dataset)) as progress_bar:
            for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
                # Setup for forward
                cw_idxs = cw_idxs.to(device)
                cc_idxs = cc_idxs.to(device)
                qw_idxs = qw_idxs.to(device)
                qc_idxs = qc_idxs.to(device)
                batch_size = cw_idxs.size(0)
                optimizer.zero_grad()

                # Forward
                log_p1, log_p2 = model(cw_idxs, cc_idxs, qw_idxs, qc_idxs)
                y1, y2 = y1.to(device), y2.to(device)
                loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
                loss_val = loss.item()

                # Backward
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step(step // batch_size)
                ema(model, step // batch_size)

                # Log info
                step += batch_size
                progress_bar.update(batch_size)
                progress_bar.set_postfix(epoch=epoch,
                                         NLL=loss_val)
                tbx.add_scalar('train/NLL', loss_val, step)
                tbx.add_scalar('train/LR',
                               optimizer.param_groups[0]['lr'],
                               step)

                steps_till_eval -= batch_size
                if steps_till_eval <= 0:
                    steps_till_eval = args.eval_steps

                    # Evaluate and save checkpoint
                    log.info('Evaluating at step {}...'.format(step))
                    ema.assign(model)
                    results, pred_dict = evaluate(model, dev_loader, device,
                                                  args.dev_eval_file,
                                                  args.max_ans_len,
                                                  args.use_squad_v2)
                    saver.save(step, model, results[args.metric_name], device)
                    ema.resume(model)

                    # Log to console
                    results_str = ', '.join('{}: {:05.2f}'.format(k, v)
                                            for k, v in results.items())
                    log.info('Dev {}'.format(results_str))

                    # Log to TensorBoard
                    log.info('Visualizing in TensorBoard...')
                    for k, v in results.items():
                        tbx.add_scalar('dev/{}'.format(k), v, step)
                    util.visualize(tbx,
                                   pred_dict=pred_dict,
                                   eval_path=args.dev_eval_file,
                                   step=step,
                                   split='dev',
                                   num_visuals=args.num_visuals)

In [8]:
def evaluate(model, data_loader, device, eval_file, max_len, use_squad_v2):
    nll_meter = util.AverageMeter()

    model.eval()
    pred_dict = {}
    with open(eval_file, 'r') as fh:
        gold_dict = json_load(fh)
    with torch.no_grad(), \
            tqdm(total=len(data_loader.dataset)) as progress_bar:
        for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in data_loader:
            # Setup for forward
            cw_idxs = cw_idxs.to(device)
            cc_idxs = cc_idxs.to(device)
            qw_idxs = qw_idxs.to(device)
            qc_idxs = qc_idxs.to(device)
            batch_size = cw_idxs.size(0)

            # Forward
            log_p1, log_p2 = model(cw_idxs, cc_idxs, qw_idxs, qc_idxs)
            y1, y2 = y1.to(device), y2.to(device)
            loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
            nll_meter.update(loss.item(), batch_size)

            # Get F1 and EM scores
            p1, p2 = log_p1.exp(), log_p2.exp()
            starts, ends = util.discretize(p1, p2, max_len, use_squad_v2)

            # Log info
            progress_bar.update(batch_size)
            progress_bar.set_postfix(NLL=nll_meter.avg)

            preds, _ = util.convert_tokens(gold_dict,
                                           ids.tolist(),
                                           starts.tolist(),
                                           ends.tolist(),
                                           use_squad_v2)
            pred_dict.update(preds)

    model.train()

    results = util.eval_dicts(gold_dict, pred_dict, use_squad_v2)
    results_list = [('NLL', nll_meter.avg),
                    ('F1', results['F1']),
                    ('EM', results['EM'])]
    if use_squad_v2:
        results_list.append(('AvNA', results['AvNA']))
    results = OrderedDict(results_list)

    return results, pred_dict

In [9]:
main(train_args)

[04.26.19 19:34:21] Args: {
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.2,
    "ema_decay": 0.999,
    "eval_steps": 50000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 100,
    "l2_wd": 0.0,
    "load_path": null,
    "lr": 0.5,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "name": "train",
    "num_epochs": 30,
    "num_visuals": 10,
    "num_workers": 4,
    "save_dir": "./save/train/train-46",
    "seed": 224,
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[04.26.19 19:34:21] Using random seed 224...
[04.26.19 19:34:21] Loading embeddings...
[04.26.19 19:34:

  0%|          | 0/129922 [00:00<?, ?it/s]

[04.26.19 19:35:25] Saver will maximize F1...
[04.26.19 19:35:25] Training...
[04.26.19 19:35:25] Starting epoch 1...


 39%|███▊      | 50048/129922 [12:05<20:22, 65.36it/s, NLL=5.87, epoch=1]

[04.26.19 19:47:31] Evaluating at step 50048...


...

[04.26.19 20:00:02] Saved checkpoint: ./save/train/train-46/step_100096.pth.tar
[04.26.19 20:00:02] Dev NLL: 04.88, F1: 52.05, EM: 52.02, AvNA: 52.36
[04.26.19 20:00:02] Visualizing in TensorBoard...


  0%|          | 0/129922 [00:00<?, ?it/s]

[04.26.19 20:07:19] Starting epoch 2...


  6%|▌         | 7552/129922 [01:51<29:24, 69.36it/s, NLL=4.63, epoch=2]
Traceback (most recent call last):
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
Traceback (most recent call last):
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessi

KeyboardInterrupt: 

In [9]:
torch.backends.cudnn.version()

7301

# посмотрим, из чего состоят cc_idxs

In [7]:
train_dataset = SQuAD(train_args.train_record_file, train_args.use_squad_v2)
train_loader = data.DataLoader(train_dataset,
                               batch_size=train_args.batch_size,
                               shuffle=True,
                               num_workers=train_args.num_workers,
                               collate_fn=collate_fn)

In [8]:
train_dataset.get_char_dict_size()

1346

In [22]:
jjj = 0
cset1 = set()
cset2 = set()
for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
    jjj += 1
#     print(cc_idxs.shape)
#     print(cc_idxs.numpy())
    cset1 |= set(cc_idxs.numpy().ravel())
    cset2 |= set(qc_idxs.numpy().ravel())
#     print(cc_idxs[0, 1, :])
#     if jjj > 100:
#         break

In [23]:
''.join([str(x) + ' ' for x in list(cset1)])

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 245 247 248 249 250 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 2

In [24]:
''.join([str(x) + ' ' for x in list(cset2)])

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 534 535 25 538 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 523 77 78 79 80 81 594 82 76 83 86 87 88 89 84 91 604 85 90 1110 94 609 528 527 101 102 104 105 106 107 108 109 110 111 112 113 114 115 628 117 119 631 120 122 121 642 131 646 139 140 653 143 144 655 146 661 662 663 1175 1176 163 165 166 542 167 678 172 176 516 189 517 220 230 231 1274 253 259 260 262 263 1288 265 266 1289 1292 1291 270 1290 272 273 276 279 282 290 291 810 303 306 309 310 311 312 825 313 321 322 324 325 326 327 329 330 331 332 333 334 336 340 343 345 348 1372 1374 1375 353 355 356 359 360 361 366 367 373 379 384 587 589 409 590 591 1100 422 423 424 440 441 442 443 478 479 480 481 483 486 1002 498 501 '

# тест

In [22]:
aaaa = np.array([[0, 1], [1, 2]]).ravel().tolist()

In [23]:
aaaa

[0, 1, 1, 2]

In [24]:
set(aaaa)

{0, 1, 2}

In [5]:
util.get_available_devices()

(device(type='cuda', index=0), [0])

# тест2

In [6]:
device, train_args.gpu_ids = util.get_available_devices()

In [7]:
device, train_args.gpu_ids

(device(type='cuda', index=0), [0])

In [10]:
A = torch.rand((19456, 1, 16, 100)).to(device)

In [12]:
B = nn.Conv2d(in_channels = 1, 
                out_channels = 72, 
                kernel_size = (3, 100),
                padding=[1, 0]).cuda()(A)

In [13]:
B

tensor([[[[-0.2982],
          [-0.2431],
          [-0.7081],
          ...,
          [-0.7422],
          [-0.6618],
          [-0.3046]],

         [[ 0.0683],
          [-0.0804],
          [-0.0250],
          ...,
          [ 0.1863],
          [-0.1117],
          [-0.1203]],

         [[-0.3199],
          [-0.1380],
          [-0.1947],
          ...,
          [-0.0441],
          [-0.3234],
          [-0.0039]],

         ...,

         [[-0.0629],
          [ 0.1735],
          [-0.1044],
          ...,
          [-0.3480],
          [ 0.0234],
          [ 0.2862]],

         [[-0.2767],
          [-0.3573],
          [-0.6291],
          ...,
          [-0.4452],
          [-0.4202],
          [-0.4769]],

         [[ 0.2735],
          [ 0.4395],
          [ 0.2507],
          ...,
          [ 0.1173],
          [ 0.3946],
          [ 0.0975]]],


        [[[-0.2974],
          [-0.4128],
          [-0.4716],
          ...,
          [-0.4754],
          [-0.5159],
     